In [89]:
import tensorflow as tf
from iris_data import load_data, train_input_fn, eval_input_fn, SPECIES
tf.logging.set_verbosity(tf.logging.ERROR)

In [90]:
# 参数
batch_size = 100
train_steps = 1000

In [91]:
# 加载数据
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [92]:
# 特征列
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [93]:
# 选择模型
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],  # 定义每个隐层的单元数（2个隐层，每层10个隐藏单元）
    n_classes=3)  # 类别数

In [94]:
# 训练，传入数据train_x即为特征，train_y即为标签
classifier.train(
    input_fn=lambda: train_input_fn(features=train_x,
                                    labels=train_y,
                                    batch_size=batch_size),
    steps=train_steps)

In [95]:
# 评估，返回eval_result是一个字典，有4个key：accuracy，average_loss，global_step，loss
eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(features=test_x,
                                   labels=test_y,
                                   batch_size=batch_size))

print('Test set accuracy: {:0.3f}'.format(eval_result["accuracy"]))

Test set accuracy: 0.933


In [96]:
# 预测数据，3个实例
expected = ['Setosa', 'Versicolor', 'Virginica']  # 这3个实例的期望类别
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [103]:
# predictions包含所有的预测
predictions = classifier.predict(
    input_fn=lambda: eval_input_fn(features=predict_x,
                                   labels=None,
                                   batch_size=batch_size))

In [104]:
# 打印预测结果
template = 'Prediction is "{}" ({:.1f}%), expected "{}"'  # 类别，概率，期望类别

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]  # 预测的标签编号
    probability = pred_dict['probabilities'][class_id]  # 该类别的概率

    print(template.format(SPECIES[class_id], 100 * probability, expec))

Prediction is "Setosa" (99.8%), expected "Setosa"
Prediction is "Versicolor" (99.5%), expected "Versicolor"
Prediction is "Virginica" (97.7%), expected "Virginica"
